<a href="https://colab.research.google.com/github/LeeHakHyun/Tensorflow2.0_Practice/blob/master/Basic_Model/tensorflow2_0_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.0.0-beta0 


     |████████████████████████████████| 87.9MB 1.2MB/s 
     |████████████████████████████████| 3.1MB 63.2MB/s 
     |████████████████████████████████| 501kB 51.8MB/s 
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [2]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
  pass



TensorFlow 2.x selected.


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras를 임포트합니다
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 헬퍼(helper) 라이브러리를 임포트합니다
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


2.0.0-beta1


In [4]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
print(train_images.shape)
train_images = train_images.reshape((60000, 28, 28, 1))
print(train_images.shape)


4423680/4422102 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000, 28, 28, 1)


In [5]:
encoder_input  = keras.Input(shape = (28, 28, 1), name = 'img')
x              = layers.Conv2D(16, 3, strides = 2, padding='SAME', activation='relu')(encoder_input)
x              = layers.Conv2D(32, 3, strides = 2, padding='SAME', activation='relu')(x)
encoder_output = layers.Conv2D(64, 3, strides = 1, padding='SAME', activation='relu')(x)

encoder = keras.Model(encoder_input, encoder_output, name = 'encoder')
encoder.summary()



Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
Total params: 23,296
Trainable params: 23,296
Non-trainable params: 0
_________________________________________________________________


In [6]:
decoder_input  = keras.Input(shape = (7, 7, 64), name = 'encoding_img');
x              = layers.Conv2DTranspose(32, 3, strides = 2, padding='SAME', activation='relu')(decoder_input)
x              = layers.Conv2DTranspose(16, 3, strides = 2, padding='SAME', activation='relu')(x)
x              = layers.Conv2D(1, 3, strides = 1, padding='SAME', activation='relu')(x)
x              = layers.Conv2D(1, 3, strides = 1, padding='SAME', activation='relu')(x)
decoder_output = layers.Conv2D(1, 3, strides = 1, padding='SAME', activation='relu')(x)

decoder = keras.Model(decoder_input, decoder_output, name = 'decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoding_img (InputLayer)    [(None, 7, 7, 64)]        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 32)        18464     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 16)        4624      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 1)         145       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 1)         10        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 1)         10        
Total params: 23,253
Trainable params: 23,253
Non-trainable params: 0
_______________________________________________________

In [7]:
autoencoder_input = keras.Input(shape = (28, 28, 1), name = 'img')
encoding = encoder(autoencoder_input)
decoding = decoder(encoding);
autoencoder = keras.Model(autoencoder_input, decoding, name = 'autoencoder')

autoencoder.summary()


Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 28, 28, 1)]       0         
_________________________________________________________________
encoder (Model)              (None, 7, 7, 64)          23296     
_________________________________________________________________
decoder (Model)              (None, 28, 28, 1)         23253     
Total params: 46,549
Trainable params: 46,549
Non-trainable params: 0
_________________________________________________________________


In [9]:
autoencoder.compile(optimizer=keras.optimizers.Adam(), loss = 'binary_crossentropy');
autoencoder.fit(train_images, train_images, batch_size = 64, epochs = 10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 4s 70us/sample - loss: -1107.5068
Epoch 2/10
60000/60000 [==============================] - 4s 63us/sample - loss: -1107.5032
Epoch 3/10
60000/60000 [==============================] - 4s 63us/sample - loss: -1107.2573
Epoch 4/10
60000/60000 [==============================] - 4s 62us/sample - loss: -1107.2147
Epoch 5/10
60000/60000 [==============================] - 4s 63us/sample - loss: -1107.4461
Epoch 6/10
60000/60000 [==============================] - 4s 63us/sample - loss: -1107.9100
Epoch 7/10
60000/60000 [==============================] - 4s 63us/sample - loss: -1108.1498
Epoch 8/10
60000/60000 [==============================] - 4s 64us/sample - loss: -1107.7953
Epoch 9/10
60000/60000 [==============================] - 4s 65us/sample - loss: -1107.8980
Epoch 10/10
60000/60000 [==============================] - 4s 65us/sample - loss: -1107.7620


ValueError: ignored